<a href="https://colab.research.google.com/github/sundarjhu/UACJ_EngineeringWeek2021/blob/main/Astropy_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = 'Sundar Srinivasan <s.srinivasan@irya.unam.mx>'
__version__ = '20210913'
__datasets__ = ['SIMBAD']  # datos usados en este notebook
__keywords__ = ['UACJ', 'Engineering Week', '2021', 'astropy', 'Table'], # palabras clave relevantes a este notebook

# **Demonstration of the Astropy Table format**

---



---



In [ ]:
from astropy.table import Table, join
import numpy as np

## We will first construct a table with some basic information -- positions for five stars

In [ ]:
id = ['lam Vel', 'IRAS 16205-2626', 'eta Car', 'alp Cen', 'IRC +10216'] #unique identifiers for three stars
RA = [136.9989911, 245.9133333, 161.2649625, 219.8738330, 146.989193] #right ascension
DEC = [-43.4325909, -26.5547222, -59.6845167, -60.8322220, 13.2787680] #declination
basic_info = Table([id, RA, DEC], names = ('ID', 'RA', 'DEC'))
basic_info

### You can view the column names without having to view the entire file

In [ ]:
basic_info.colnames

### You can add, remove, or rename the columns

In [ ]:
temp = basic_info.copy() #copying to a temporary table that we'll mess around with.
temp['facility'] = ['Gaia', '2MASS', 'Gaia', 'Gaia', 'Gaia'] #adding a column
temp.pprint()
temp.remove_columns(['RA', 'DEC'])
temp.pprint()
temp.rename_columns(['facility'], ['telescope'])
temp.pprint()

## Construct another table, this one with distances and proper motions for three stars

In [ ]:
id = ['V838 Mon', 'alp Cen', 'IRC +10216']
d = [5900, 1.34, 90] # distances in pc
PM_RA = [-0.536, -3608, 33.84] # proper motion component along RA in milliarcsec/yr
PM_DEC = [0.078, 686, 10.0] # proper motion component along DEC in milliarcsec/yr
other_info = Table([id, d, PM_RA, PM_DEC], names = ('ID', 'distance', 'PM_RA', 'PM_DEC'))
other_info

ID,distance,PM_RA,PM_DEC
str10,float64,float64,float64
V838 Mon,5900.0,-0.536,0.078
alp Cen,1.34,-3608.0,686.0
IRC +10216,90.0,33.84,10.0


## Let's investigate various types of joins using these two tables

### An inner join is like an intersection -- the resulting table will only contain sources that are present in both tables.

In [ ]:
inner_join = join(basic_info, other_info, keys = ['ID'], join_type = 'inner')
inner_join

### A left join will result in a table with all the rows in the first ("left") table. If a row from this table has a match in the second ("right") table, it will be shown. If not, the corresponding fields will be empty.

In [ ]:
left_join = join(basic_info, other_info, keys = ['ID'], join_type = 'left')
left_join

### A right join is similar to a left join, but will contain all rows from the second ("right") table.

In [ ]:
right_join = join(basic_info, other_info, keys = ['ID'], join_type = 'right')
right_join

## A column with missing fields is represented as a MaskedColumn, making it easier to isolate rows with missing data or to separate them from rows with data.

In [ ]:
left_join['distance'].mask

### Here, we isolate the rows of left_join whose distances are unavailable

In [ ]:
k = np.nonzero(left_join['distance'].mask)
print(left_join[k])

### Conversely, we can isolate the rows of left_join with valid distances

In [ ]:
k = np.nonzero(~left_join['distance'].mask)
print(left_join[k])

## To save tables to file, we will use the Virtual Observatory Table (VOT) format

In [ ]:
basic_info.write("basic_info.vot", format = 'votable', overwrite = True)
#You can also write a subset (rows and/or columns) to file as follows:
#basic_info[2:4].write("basic_info.vot", format = 'votable', overwrite = True) #this writes a two-row table to file
#basic_info['ID', 'RA'].write("basic_info.vot", format = 'votable', overwrite = True) #this does not write the DEC column to file.

## Reading from a VOTable is also easy

In [ ]:
b = Table.read("basic_info.vot", format = 'votable')
b

# Assignment: read the [astropy documentation](https://docs.astropy.org/en/stable/table/index.html) (or use numpy) to solve the following:

In [ ]:
galaxy = ['NGC 6822', 'M 87', 'M 33', 'NGC 6822', 'NGC 6822', 'M 33']
distance = [586, 53500, 1020, 495, 540, 935] # distance in kpc
data = Table([galaxy, distance], names = ('Galaxy', 'Distance'))

# Sort this table such that rows for the same galaxy appear together and the entries are in increasing order of distance.